<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generating-Word-Embeddings---Lab" data-toc-modified-id="Generating-Word-Embeddings---Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generating Word Embeddings - Lab</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Getting-Started" data-toc-modified-id="Getting-Started-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Getting Started</a></span></li><li><span><a href="#Preparing-the-Data" data-toc-modified-id="Preparing-the-Data-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Preparing the Data</a></span></li><li><span><a href="#Training-the-Model" data-toc-modified-id="Training-the-Model-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Training the Model</a></span></li><li><span><a href="#Examining-Our-Word-Vectors" data-toc-modified-id="Examining-Our-Word-Vectors-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Examining Our Word Vectors</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Summary</a></span></li></ul></li></ul></div>

# Generating Word Embeddings - Lab

## Introduction

In this lab, we'll learn how to generate our own word embeddings by training our own Word2Vec model, and also by building embedding layers right into our Deep Neural Networks!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, we'll start by creating our own word embeddings by making use of the Word2Vec Model. Then, we'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in our model. 

The easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which we can use to begin training immediately. For this lab, we'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.  In this lab, we'll learn how to train a Word2Vec model on the text data to generate word embeddings for them. In the next lab, we'll then use the vectors created by our Word2Vec model to effectively train a classifier to predict the category of news given the headline and description of each article. In this lab, we won't do any classification, although we will learn how to train a Word2Vec model and explore the relationships between different word vectors in our embedding!

Run the cell below to import everything we'll need for this lab. 

In [3]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, we'll import the data. You'll find the data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to also include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what our data looks like. 

In [4]:
raw_df = pd.read_json('News_Category_Dataset_v2.zip', compression='zip', lines=True)
raw_df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since we're working with text data, we'll still need to do some basic preprocessing and tokenize our data. You'll notice from the sample of the data above that two different columns contain text data--`headline` and `short_description`. The more text data our Word2Vec model has, the better it will perform. Therefore, we'll want to combine the two columns before tokenizing each comment and training our Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/stephanosterburg/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
raw_df['combined_text'] = raw_df.headline + ' ' + raw_df.short_description
data = raw_df['combined_text'].map(word_tokenize)

Let's inspect the first 5 items in `data` to see how everything looks. 

In [12]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

You'll notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Recall from our previous lesson on how Word2Vec works that we can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If our window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example we saw).

Now that we've prepared our data, let's train our model and explore a bit!

## Training the Model

We'll start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [13]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that we've created our Word2Vec model, we still need to train it on our model. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which we can find in `model.corpus_count`. 
    * The number of `epochs` we want to train for, which we'll set to `10`

In [14]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55563192, 67352790)

Great! We now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which we'll find stored inside of `model.wv`. For simplicity's sake, let's go ahead and store this inside of the variable `wv` in order to save ourselves some keystrokes down the line. 

In [15]:
wv = model.wv

## Examining Our Word Vectors

Now that we have a trained Word2Vec model, let's go ahead and explore the relationships between some of the words in our corpus! 

One cool thing we can use Word2Vec for is to get the most similar words to a given word. We can do this passing in the word to `wv.most_similar()`. 

In the cell below, let's try getting the most similar word to `'Texas'`.

In [16]:
wv.most_similar('Texas')

[('Ohio', 0.817565381526947),
 ('Connecticut', 0.8128525614738464),
 ('Maryland', 0.8109374642372131),
 ('Louisiana', 0.8096040487289429),
 ('Georgia', 0.8024595975875854),
 ('Pennsylvania', 0.7995883226394653),
 ('Florida', 0.7935335040092468),
 ('Oregon', 0.7923513650894165),
 ('Colorado', 0.7891526222229004),
 ('Michigan', 0.7891079187393188)]

Interesting! All of the most similar words are also states. 

We can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [17]:
wv.most_similar(negative='Texas')

[('Headstrong', 0.5054244995117188),
 ('Parent/Grandparent', 0.46355846524238586),
 ('Profit-Minded', 0.423515260219574),
 ('Likened', 0.40306898951530457),
 ('Anti-Gang', 0.38777196407318115),
 ('UIUC', 0.3867909908294678),
 ('Gang-Raping', 0.3778834342956543),
 ('writer-producer', 0.3699330985546112),
 ('maitre', 0.36783236265182495),
 ('much-vaunted', 0.36198359727859497)]

These seem like just noise. This is because of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with our given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

We can also get the vector for a given word by passing in the word as if we were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [18]:
wv['Texas']

array([-0.90574086, -1.8432653 ,  1.1637048 , -1.4141514 , -2.446556  ,
        2.6632369 ,  0.5363231 ,  0.6434574 , -0.16886306,  1.2452811 ,
        0.18157928,  0.6827276 , -0.5813451 , -2.1217968 ,  1.5743381 ,
        0.9466911 , -0.77030396, -0.41236237,  0.01473982,  1.9518025 ,
        1.384515  , -1.2575916 , -0.15106808,  1.9642444 ,  1.3096946 ,
       -0.56771344, -0.05523844,  0.7435967 , -0.78647125, -2.3750017 ,
        2.838864  ,  0.5471862 , -1.6648989 ,  0.13019489, -0.3500942 ,
        1.1589749 , -0.72425383, -2.2980154 ,  1.7246414 , -2.7293758 ,
       -1.1440101 ,  0.6571775 , -0.7514698 , -1.9363602 , -1.4016675 ,
       -0.5605644 , -0.48543313, -1.1021756 , -1.3027012 , -1.9391624 ,
       -0.28206152,  0.21828054, -0.10429979, -1.3635684 , -0.7016807 ,
        0.2672464 , -1.6459266 ,  1.3060112 ,  0.01555841,  1.887298  ,
       -0.9830562 , -0.50394565,  0.64302635,  0.07018988,  1.7980229 ,
       -0.01329768, -0.66503024,  0.84516984, -0.5609635 ,  0.79

Let's get all of the word vectors from the object at once. We can find these inside of `wv.vectors`.  Do this now in the cell below.  

In [19]:
wv.vectors

array([[ 4.1655872e-02,  1.9887875e+00, -1.7322874e+00, ...,
        -1.1416301e+00,  5.7626468e-01, -4.5578378e-01],
       [ 1.5089579e+00,  3.8187981e+00, -3.8114100e+00, ...,
         3.1596801e-01,  5.2471936e-01, -1.1829472e-01],
       [ 1.4504300e+00,  1.9349716e+00, -4.2482562e+00, ...,
         9.3261421e-01, -5.0566041e-01,  8.5481894e-01],
       ...,
       [ 1.2020010e-01, -3.5165567e-02,  8.5988089e-02, ...,
        -2.3682166e-02,  8.0137968e-02,  2.3178565e-03],
       [ 4.9732132e-03, -1.9521333e-02, -3.0272631e-02, ...,
        -2.0161964e-02,  4.6132699e-02,  3.0515183e-02],
       [ 5.4112092e-02,  3.4975927e-02,  5.0496612e-02, ...,
        -3.2302909e-02,  6.7497015e-02, -2.9208483e-02]], dtype=float32)

As a final exercise, let's try recreating the _'king' - 'man' + 'woman' = 'queen'_ example we've seen before. We can do this by using the `most_similar` function and putting the things we want added together inside of an array passed to the `positive` parameter, and the things we want subtracted as an array passed to the the `negative` parameter. 

Do this now in the cell below. 

In [20]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('princess', 0.6334392428398132),
 ('revival', 0.6280776262283325),
 ('goddess', 0.6166364550590515),
 ('monster', 0.6041973829269409),
 ('title', 0.5996745824813843),
 ('dancer', 0.5989830493927002),
 ('brunette', 0.5989501476287842),
 ('crown', 0.5914324522018433),
 ('villain', 0.587134599685669),
 ('spinoff', 0.5756184458732605)]

As we can see from the output above, our model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As we can see from the word in first place, 'reminiscent', our model is far from perfect. This is likely because we didn't give it too much training, or training data. However, for the small amount of training data it was given, the model still performs remarkably well! 

We'll see in the next lab that from a practical standpoint, one of the best things we can do for performance is to start by loading in the weights from an open-sourced model that has been trained for a very long time on a massive amount of data, such as the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless our text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, we learned how to train and use a Word2Vec model to created vectorized word embeddings!